In [2]:
 # Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
#### from us import states
import gmaps

#Census API Key
from config import census_api_key
c = Census(census_api_key, year=2016)

# gmaps key
from config import g_key
gmaps.configure(api_key=g_key)

In [3]:
# Run Census Search to retrieve data on all zip codes (2016 ACS5 Census)
### retrieve and create Data Frame
census_data = c.acs5.get(("NAME","B01003_001E"), {'for': 'zip code tabulation area:*'})

census_pd = pd.DataFrame(census_data)
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "NAME": "Name",
                                      "zip code tabulation area": "Zipcode"})

# Final DataFrame
census_pd = census_pd[["Zipcode", "Population"]]

# Visualize
print(len(census_pd))
census_pd.head()

33120


,Zipcode,Population
0,99119,1209.0
1,99128,209.0
2,99141,5140.0
3,99149,180.0
4,99153,390.0


In [4]:
#Save to csv for later use
census_pd.to_csv("census_data_zipcodes.csv", encoding="utf-8", index=False)

In [5]:
#checking tables
c.acs5.tables()

[{'name': 'B18104',
  'description': 'SEX BY AGE BY COGNITIVE DIFFICULTY',
  'variables': 'https://api.census.gov/data/2016/acs/acs5/groups/B18104.json'},
 {'name': 'B17015',
  'description': 'POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILIES BY FAMILY TYPE BY SOCIAL SECURITY INCOME BY SUPPLEMENTAL SECURITY INCOME (SSI) AND CASH PUBLIC ASSISTANCE INCOME',
  'variables': 'https://api.census.gov/data/2016/acs/acs5/groups/B17015.json'},
 {'name': 'B18105',
  'description': 'SEX BY AGE BY AMBULATORY DIFFICULTY',
  'variables': 'https://api.census.gov/data/2016/acs/acs5/groups/B18105.json'},
 {'name': 'B17016',
  'description': 'POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILIES BY FAMILY TYPE BY WORK EXPERIENCE OF HOUSEHOLDER AND SPOUSE',
  'variables': 'https://api.census.gov/data/2016/acs/acs5/groups/B17016.json'},
 {'name': 'B18106',
  'description': 'SEX BY AGE BY SELF-CARE DIFFICULTY',
  'variables': 'https://api.census.gov/data/2016/acs/acs5/groups/B18106.json'},
 {'name': 'B17017',
  '

In [6]:
# call texas DF for zipcodes and lat lng
texas_csvpath =('reduced_texas_df.csv')
texas_df = pd.read_csv(texas_csvpath)
texas_df.head()

,Unnamed: 0,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Description,City,State,...,Weather_Timestamp,Temperature(F),Visibility(mi),Wind_Speed(mph),Precipitation(in),Weather_Condition,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,261002,201.0,2,2016-11-30 16:03:54,2016-11-30 17:20:00,30.336502,-97.755646,Accident on RM-2222 Northland Dr at Mo Pac Exp...,Austin,TX,...,2016-11-30 15:51:00,66.0,10.0,5.8,NaN,Clear,Day,Day,Day,Day
1,261003,201.0,2,2016-11-30 16:32:18,2016-11-30 17:47:02,30.328165,-97.694305,Accident on Grand Canyon Dr between Westheimer...,Austin,TX,...,2016-11-30 16:51:00,62.1,10.0,4.6,NaN,Clear,Day,Day,Day,Day
2,261004,201.0,2,2016-11-30 16:31:45,2016-11-30 17:46:34,30.326077,-97.692307,Accident on Saint Johns Ave at Cameron Rd.,Austin,TX,...,2016-11-30 16:51:00,62.1,10.0,4.6,NaN,Clear,Day,Day,Day,Day
3,261005,201.0,2,2016-11-30 16:06:49,2016-11-30 17:27:00,32.856014,-97.513084,#2 lane blocked and right hand shoulder blocke...,Fort Worth,TX,...,2016-11-30 16:00:00,NaN,NaN,NaN,NaN,NaN,Day,Day,Day,Day
4,261006,201.0,3,2016-11-30 16:35:01,2016-11-30 17:49:39,32.797668,-97.212090,#2 lane blocked and right hand shoulder blocke...,Fort Worth,TX,...,NaN,NaN,NaN,NaN,NaN,NaN,Day,Day,Day,Day


In [7]:
# checking DF columns
texas_df.columns

Index(['Unnamed: 0', 'TMC', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat',
       'Start_Lng', 'Description', 'City', 'State', 'Zipcode',
       'Weather_Timestamp', 'Temperature(F)', 'Visibility(mi)',
       'Wind_Speed(mph)', 'Precipitation(in)', 'Weather_Condition',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')

In [8]:
# getting rid of columns 
red_texas_df = texas_df.loc[:,["Zipcode","Start_Lat","Start_Lng","City","State"]]
red_texas_df.head()


,Zipcode,Start_Lat,Start_Lng,City,State
0,78731,30.336502,-97.755646,Austin,TX
1,78752-2826,30.328165,-97.694305,Austin,TX
2,78752,30.326077,-97.692307,Austin,TX
3,76135,32.856014,-97.513084,Fort Worth,TX
4,76118,32.797668,-97.212090,Fort Worth,TX


In [9]:
# checking how many per zipcode
zip_groups = red_texas_df.groupby("Zipcode")
zip_groups["Zipcode"].count()

Zipcode
75001         38
75001-3434     1
75001-4302     1
75001-4531     1
75001-4601     2
              ..
79938-8900     1
79938-9118     3
79938-9604     5
79938-9613     5
79938-9622     1
Name: Zipcode, Length: 42054, dtype: int64

In [10]:
# split and create new zipcade columns
new = red_texas_df["Zipcode"].str.split("-", n = 1, expand = True)
red_texas_df["New Zipcode"]=new[0]
red_texas_df.head()

,Zipcode,Start_Lat,Start_Lng,City,State,New Zipcode
0,78731,30.336502,-97.755646,Austin,TX,78731
1,78752-2826,30.328165,-97.694305,Austin,TX,78752
2,78752,30.326077,-97.692307,Austin,TX,78752
3,76135,32.856014,-97.513084,Fort Worth,TX,76135
4,76118,32.797668,-97.212090,Fort Worth,TX,76118


In [11]:
zip_groups =red_texas_df.groupby("New Zipcode")
zip_groups.count()

,Zipcode,Start_Lat,Start_Lng,City,State
New Zipcode,,,,,
75001,62,62,62,62,62
75002,21,21,21,21,21
75006,1251,1251,1251,1251,1251
75007,222,222,222,222,222
75009,1,1,1,1,1
...,...,...,...,...,...
79932,3,3,3,3,3
79934,98,98,98,98,98
79935,355,355,355,355,355


In [12]:
# Drop old zipcode and Not zipcode
red_texas_df = red_texas_df.loc[:,["New Zipcode","Start_Lat","Start_Lng","City","State"]]
red_texas_df.head()

,New Zipcode,Start_Lat,Start_Lng,City,State
0,78731,30.336502,-97.755646,Austin,TX
1,78752,30.328165,-97.694305,Austin,TX
2,78752,30.326077,-97.692307,Austin,TX
3,76135,32.856014,-97.513084,Fort Worth,TX
4,76118,32.797668,-97.212090,Fort Worth,TX


In [13]:
red_texas_df.columns


Index(['New Zipcode', 'Start_Lat', 'Start_Lng', 'City', 'State'], dtype='object')

In [14]:
# rename new zip to zip and start lat and lng to Latitude and longitude
red_texas_df = red_texas_df.rename(columns={"New Zipcode":"Zipcode","Start_Lat":"Latitude","Start_Lng":"Longitude"})
red_texas_df.head()

,Zipcode,Latitude,Longitude,City,State
0,78731,30.336502,-97.755646,Austin,TX
1,78752,30.328165,-97.694305,Austin,TX
2,78752,30.326077,-97.692307,Austin,TX
3,76135,32.856014,-97.513084,Fort Worth,TX
4,76118,32.797668,-97.212090,Fort Worth,TX


In [15]:
# merge pop and texas df for Zipcode DF 
zipcode_df = pd.merge(red_texas_df,census_pd, on="Zipcode", how="left")
zipcode_df.head()

,Zipcode,Latitude,Longitude,City,State,Population
0,78731,30.336502,-97.755646,Austin,TX,27401.0
1,78752,30.328165,-97.694305,Austin,TX,19902.0
2,78752,30.326077,-97.692307,Austin,TX,19902.0
3,76135,32.856014,-97.513084,Fort Worth,TX,22878.0
4,76118,32.797668,-97.212090,Fort Worth,TX,16002.0


In [16]:
# clean zipcode df for map use and save csv for possible later use - 

clean_texas_df = zipcode_df.dropna(how='any')
clean_texas_df.to_csv("clean_texas_df.csv", index=False)

clean_texas_df

,Zipcode,Latitude,Longitude,City,State,Population
0,78731,30.336502,-97.755646,Austin,TX,27401.0
1,78752,30.328165,-97.694305,Austin,TX,19902.0
2,78752,30.326077,-97.692307,Austin,TX,19902.0
3,76135,32.856014,-97.513084,Fort Worth,TX,22878.0
4,76118,32.797668,-97.212090,Fort Worth,TX,16002.0
...,...,...,...,...,...,...
376440,77027,29.749589,-95.456110,Houston,TX,15498.0
376441,77007,29.778089,-95.373146,Houston,TX,35185.0
376442,77020,29.774190,-95.320940,Houston,TX,25908.0
376443,77396,29.941686,-95.295187,Humble,TX,50196.0


In [27]:
 # Store 'Lat' and 'Lng' into  locations  create WEight for pop had to divide by 1000 to sgrink the numbers
locations = clean_texas_df[["Latitude", "Longitude"]].astype(float)
population = (clean_texas_df["Population"]/1000).tolist()
accidents = clean_texas_df["Zipcode"].value_counts()

In [18]:
# Create a population Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=population, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = .17)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [34]:
# checking counts
accidents

78753    5487
75243    4877
78704    4567
78741    4045
78744    4020
         ... 
76528       1
76878       1
75752       1
75844       1
78119       1
Name: Zipcode, Length: 1224, dtype: int64

In [57]:
# creating seperate DF to not make sure doesnt change original
accidents.to_frame()
accidents_df = accidents.reset_index()
clean_accidents_df= accidents_df.loc[accidents_df["Zipcode"] > 10]
sort_accidents_df = clean_accidents_df.sort_values(['Zipcode'], ascending=False)
top_20_df=sort_accidents_df.loc[0:20]
ren_top_20_df=top_20_df.rename(columns={"index":"Zipcode","Zipcode":"Accident Count"})
ren_top_20_df.head()

,Zipcode,Accident Count
0,78753,5487
1,75243,4877
2,78704,4567
3,78741,4045
4,78744,4020


In [56]:
lat_lng_df = clean_texas_df.loc[:,["Zipcode","Latitude","Longitude"]]
# lat_lng_df.head()
#Drop all nunique zipcodes
merge_markers_df = pd.merge(ren_top_20_df,lat_lng_df, on="Zipcode", how="left")
merge_markers_df.drop_duplicates(subset = ['Zipcode'], keep = 'first', inplace = True)
merge_markers_df

,Zipcode,Accident Count,Latitude,Longitude
0,78753,5487,30.400223,-97.673439
5487,75243,4877,32.922962,-96.718018
10364,78704,4567,30.237190,-97.754929
14931,78741,4045,30.216488,-97.750664
18976,78744,4020,30.163210,-97.725838
22996,75228,3817,32.864021,-96.661140
26813,77002,3799,29.769836,-95.365616
30612,75207,3782,32.766266,-96.803719
34394,78745,3630,30.215410,-97.797661
38024,78759,3115,30.426615,-97.753166


In [63]:
# create info box template
info_box_template = """
<dl>
<dt>Zipcode</dt><dd>{Zipcode}</dd>
<dt>Accident Count</dt><dd>{Accident}</dd>
</dl>
"""

accident_list = []
for index, row in merge_markers_df.iterrows():
  Accident = row['Accident Count']
  lat = row['Latitude']
  lng = row['Longitude']
  Zipcode = row['Zipcode']
  print(info_box_template.format(Zipcode=Zipcode, Accident=Accident))
  accident_list.append(info_box_template.format(Zipcode=Zipcode, Accident=Accident))




<dl>
<dt>Zipcode</dt><dd>78753</dd>
<dt>Accident Count</dt><dd>5487</dd>
</dl>


<dl>
<dt>Zipcode</dt><dd>75243</dd>
<dt>Accident Count</dt><dd>4877</dd>
</dl>


<dl>
<dt>Zipcode</dt><dd>78704</dd>
<dt>Accident Count</dt><dd>4567</dd>
</dl>


<dl>
<dt>Zipcode</dt><dd>78741</dd>
<dt>Accident Count</dt><dd>4045</dd>
</dl>


<dl>
<dt>Zipcode</dt><dd>78744</dd>
<dt>Accident Count</dt><dd>4020</dd>
</dl>


<dl>
<dt>Zipcode</dt><dd>75228</dd>
<dt>Accident Count</dt><dd>3817</dd>
</dl>


<dl>
<dt>Zipcode</dt><dd>77002</dd>
<dt>Accident Count</dt><dd>3799</dd>
</dl>


<dl>
<dt>Zipcode</dt><dd>75207</dd>
<dt>Accident Count</dt><dd>3782</dd>
</dl>


<dl>
<dt>Zipcode</dt><dd>78745</dd>
<dt>Accident Count</dt><dd>3630</dd>
</dl>


<dl>
<dt>Zipcode</dt><dd>78759</dd>
<dt>Accident Count</dt><dd>3115</dd>
</dl>


<dl>
<dt>Zipcode</dt><dd>78701</dd>
<dt>Accident Count</dt><dd>3113</dd>
</dl>


<dl>
<dt>Zipcode</dt><dd>78758</dd>
<dt>Accident Count</dt><dd>3030</dd>
</dl>


<dl>
<dt>Zipcode</dt><dd>77

In [66]:
# Create a marker_layer using the zipcode list to fill in info box group by count and city?
marker_locations = merge_markers_df[["Latitude", "Longitude"]]
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,info_box_content=accident_list)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [72]:
 ##### Create a combined map
fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(markers)
plt.savefig('heat_pop_zipcode_map.png', dpi=400)
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

In [73]:
# Plot as a Terrain map
fig = gmaps.figure(map_type="TERRAIN")
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=population, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=.01)
fig.add_layer(markers)
fig.add_layer(heat_layer)
plt.savefig('terrain_heat_zipcode_map.png', dpi=400)
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

In [74]:
# decide which hypothesis test to use and calculate
